In [2]:
#Import all packages

import csv
import math
import os, sys
import sys
import numpy as np
from scipy import signal
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import librosa
import IPython.display as ipd
%matplotlib inline

In [17]:
#Question 1
'''Implement the template based chord recognition algorithm. Define a function that takes as input the
path to a wav file and returns the estimated chords sequence labels.
The output must be a list

λ_pred = [λ_pred0, λpred_1, ..., λpred_N−1] (1)

where each element λpred_n is the predicted chord label for the time frame n. 
The length of the list depends on the feature rate, i.e., both on the window length and hop size 
used for the chromagram computation and on the downsampling factor, if feature downsampling is 
applied.

The chord templates to be considered are the major triads and the minor triads, leading to a total of 24 templates.
Once the function is defined in the notebook, test the function on the wav file Beatles LetItBe.wav,
available in /data/wav/ folder, and print or plot the output.

Explain in the report the idea behind the template-based chord recognition algorithm and detail each
step implemented in the code, including pre processing and post processing phases.
'''
template_cmaj = np.array([[1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]]).T
template_cmin = np.array([[1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]]).T

"""" Function use to generate the template matrix """
def generate_template_matrix(templates):
    
    template_matrix = np.zeros((12, 12 * templates.shape[1]))

    for shift in range(12):
        #np.roll: roll array elements along a given axis.
        template_matrix[:, shift::12] = np.roll(templates, shift, axis=0)

    return template_matrix

templates = generate_template_matrix(np.concatenate((template_cmaj, template_cmin), axis=1))

"""" Labels for draw the chroma matrix """

chroma_label = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

chord_label_maj = chroma_label
chord_label_min = [s + 'm' for s in chroma_label]
chord_labels = chord_label_maj + chord_label_min


"""Normalizes the columns of a feature sequence"""
def normalize_feature_sequence(X, norm='2', threshold=0.0001, v=None):
    
    K, N = X.shape
    X_norm = np.zeros((K, N))

    if norm == '1':
        if v is None:
            v = np.ones(K, dtype=np.float64) / K
        for n in range(N):
            s = np.sum(np.abs(X[:, n]))
            if s > threshold:
                X_norm[:, n] = X[:, n] / s
            else:
                X_norm[:, n] = v

    if norm == '2':
        if v is None:
            v = np.ones(K, dtype=np.float64) / np.sqrt(K)
        for n in range(N):
            s = np.sqrt(np.sum(X[:, n] ** 2))
            if s > threshold:
                X_norm[:, n] = X[:, n] / s
            else:
                X_norm[:, n] = v
                
    if norm == 'max':
        if v is None:
            v = np.ones(K)
        for n in range(N):
            s = np.max(np.abs(X[:, n]))
            if s > threshold:
                X_norm[:, n] = X[:, n] / s
            else:
                X_norm[:, n] = v

    return X_norm

#from scipy import signal

def smooth_downsample_feature_sequence(X, Fs, filt_len=41, down_sampling=10, w_type='boxcar'):
   
    filt_kernel = np.expand_dims(signal.get_window(w_type, filt_len), axis=0)
    X_smooth = signal.convolve(X, filt_kernel, mode='same') / filt_len
    X_smooth = X_smooth[:, ::down_sampling]
    Fs_feature = Fs / down_sampling
    return X_smooth, Fs_feature

def analysis_template_match(chromagram, templates, smoothing_window_length=None, smoothing_down_sampling=None,
                            Fs=22050,
                            norm_chromagram='2', norm_output='2'):
    
    chroma_normalized = normalize_feature_sequence(chromagram, norm=norm_chromagram)
    
    if smoothing_window_length and smoothing_down_sampling:
        chroma_normalized, Fs_feat = smooth_downsample_feature_sequence(chroma_normalized, 
                                                                        down_sampling=smoothing_down_sampling,
                                                                        filt_len=smoothing_window_length,
                                                                        Fs=Fs)
        
    templates_normalized = normalize_feature_sequence(templates, norm=norm_chromagram)
    
    chord_similarity = np.matmul(templates_normalized.T, chroma_normalized)
    
    if norm_output:
         chord_similarity = normalize_feature_sequence(chord_similarity, norm=norm_output)
    
    chord_max = (chord_similarity == chord_similarity.max(axis=0)).astype(int)

    return chord_similarity, chord_max
''''def chordList(templates, template_sequence):
    input_list = [] 

    for col in range(chords_max.shape[1]-1): 
        if 1 in chords_max[:,col]: 
            index = np.where(chords_max[:,col] == 1)[0][0] 
            input_list.append(chords[index])
    return input_list'''

def chordList(templates, template_sequence):
    template_sequence[template_sequence != 0] = 1 #evidenzia bene le note nella matrice in uscita
    chords = [] #inizializza la lista degli accordi
    
    #analizza ogni campione della matrice dell'audio e di quella di riferimento mettendoli a confronto
    #quando i campioni corrispondono assegna il nome dell'accordo all'array chords
    for i in range(template_sequence.shape[1]):
        for j in range(templates.shape[1]):
            if(np.array_equal(template_sequence[:,i], templates[:,j])): 
                #majorChords
                if(j==0):
                    chords.append("C")
                if(j==1):
                    chords.append("C#")
                if(j==2):
                    chords.append("D")
                if(j==3):
                    chords.append("D#")
                if(j==4):
                    chords.append("E")
                if(j==5):
                    chords.append("F")
                if(j==6):
                    chords.append("F#")
                if(j==7):
                    chords.append("G")
                if(j==8):
                    chords.append("G#")
                if(j==9):
                    chords.append("A")
                if(j==10):
                    chords.append("A#")   
                if(j==11):
                    chords.append("B")
                    
                #minorChords
                if(j==12):
                    chords.append("Cm")
                if(j==13):
                    chords.append("C#m")
                if(j==14):
                    chords.append("Dm")
                if(j==15):
                    chords.append("D#m")
                if(j==16):
                    chords.append("Em")
                if(j==17):
                    chords.append("Fm")
                if(j==18):
                    chords.append("F#m")
                if(j==19):
                    chords.append("Gm")
                if(j==20):
                    chords.append("G#m")
                if(j==21):
                    chords.append("Am")
                if(j==22):
                    chords.append("A#m")   
                if(j==23):
                    chords.append("Bm")
    return chords

'''
def template_based_chord_recognigtion(audioPath, L=None, D=None, norm = '2', H = N):
    templates = generate_template_matrix(np.concatenate((template_cmaj, template_cmin), axis=1))
  
    #Parameters
    Fs = 22050
    N = Fs
    #N = 1024
    H = H
    #H = int(N/H_div)
    #H = 2048
    
    #Read audio
    x, Fs = librosa.load(audioPath, sr=Fs)
    
    #Algorithm
    X = librosa.stft(x, n_fft=N, hop_length=H, pad_mode='constant', center=True)
    X = np.abs(X) ** 2
    
     #logarithmic compression
    gamma = 0.1
    X = np.log(1 + gamma * np.abs(X) ** 2)
    
    #chromagram
    C = librosa.feature.chroma_stft(S=X, sr=Fs, tuning=0, hop_length=H, n_fft=N, norm=None)
    
    #analize chords
    chords_sim, chords_max = analysis_template_match(C, templates, 
                                                smoothing_window_length=L,
                                                smoothing_down_sampling=D, 
                                                Fs=Fs,
                                                norm_chromagram='2', 
                                                norm_output='max')
    # Compute normalized binary templates of analysis
    templates_normalized = normalize_feature_sequence(templates, norm=norm)
    # by multipling the most probable chords by the templates (normalized), you obtain the sequence of chords chroma
    # by chroma
    
    #matrice contentente gli array degli accordi in binario
    template_sequence = np.matmul(templates_normalized, chords_max)
    #template_sequence_del = np.delete(template_sequence, template_sequence.shape[1] - 1, axis=1) #per far combaciare le dimensioni delle matrici

    #crea la matrice degli accordi in stringhe
    #chords = chordList(templates, template_sequence_del)
    chords = chordList(templates, template_sequence)
    
    #return chords, template_sequence_del, templates, Fs, N, H
    return chords, template_sequence, templates, Fs, N, H

#Execute function
audioWav = os.path.join('data','wav', 'Beatles_LetItBe.wav')#read the path
chords_WAV, template_sequence, templates, Fs, N, H = template_based_chord_recognigtion(audioWav)#ritorna gli accordi

#Plot
#chroma_label = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
#chord_label_maj = chroma_label
#chord_label_min = [s + 'm' for s in chroma_label]
#chord_labels = chord_label_maj + chord_label_min

print(chords_WAV, len(chords_WAV))

def chordList(templates, template_sequence):
    input_list = [] 

    for col in range(chords_max.shape[1]-1): 
        if 1 in chords_max[:,col]: 
            index = np.where(chords_max[:,col] == 1)[0][0] 
            input_list.append(chords[index])
    return input_list

'''

def template_based_chord_recognigtion(audioPath, L=None, D=None, norm = '2', N = 22050, H = 22050):
    #Read audio
    #x, Fs = librosa.load(audioPath, sr=Fs)
    # load wav file
    #fn_wav = os.path.join('data', 'audio', 'Beatles_LetItBe.wav')
    #Parameters
    Fs = 22050 
    
    if(N != Fs):
        N = N
    elif(N == Fs):
        N = Fs
        
    if(H != N):
        H = H
    elif(H == N):
        H = N

    x, Fs = librosa.load(audioPath, sr=Fs)
    notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B', 'Cm', 'C#m', 'Dm', 'D#m', 'Em', 'Fm', 'F#m', 'Gm', 'G#m', 'Am', 'A#m', 'Bm'];


    X = librosa.stft(x, n_fft=N, hop_length=H, pad_mode='constant', center=True)

    X = np.abs(X) ** 2
        # a possible additional step is to use logarithmic compression 
    gamma = 0.1
    X = np.log(1 + gamma * np.abs(X) ** 2)

    C = librosa.feature.chroma_stft(S=X, sr=Fs, tuning=0, hop_length=H, n_fft=N, norm=None)


    chords_sim, chords_max = analysis_template_match(C, templates, 
                                                         smoothing_window_length=None,
                                                         smoothing_down_sampling=None,
                                                         norm_chromagram='2', 
                                                         norm_output='max')

    # Compute normalized binary templates of analysis
    templates_normalized = normalize_feature_sequence(templates, norm='2')

    # by multipling the most probable chords by the templates (normalized), you obtain the sequence of chords chroma
    # by chroma

    template_sequence = np.matmul(templates_normalized, chords_max)
    chords = chordList(templates, template_sequence)
    
    return chords, template_sequence, templates, Fs, N, H
    

audioWav = os.path.join('data','wav', 'Beatles_LetItBe.wav')
chords_WAV, template_sequence, templates, Fs, N, H = template_based_chord_recognigtion(audioWav)
print('Audio: ' '\033[1m' + 'Beatles_LetItBe.wav' + '\033[0m')
print('List of predicted chords: ' + str(len(chords_WAV))+ ' elements.\n')
print(chords_WAV)

Audio: Beatles_LetItBe.wav
List of predicted chords: 243 elements.

['C', 'C', 'G', 'G', 'Am', 'C', 'F', 'C', 'C', 'G', 'F', 'Em', 'C', 'C', 'C', 'G', 'G', 'C#m', 'Dm', 'F', 'Em', 'E', 'G', 'Dm', 'C', 'C', 'C', 'C', 'G', 'D', 'D', 'D', 'Dm', 'Em', 'E', 'G', 'Bm', 'C', 'Em', 'C', 'Em', 'Em', 'A#', 'Am', 'C', 'C', 'Em', 'Em', 'G', 'Dm', 'F', 'C', 'C', 'Am', 'Gm', 'Dm', 'E', 'C#', 'Dm', 'E', 'E', 'G', 'Dm', 'Dm', 'C', 'C', 'C', 'Gm', 'D', 'E', 'G', 'Dm', 'E', 'E', 'G', 'Bm', 'C', 'C', 'Gm', 'Am', 'C', 'Em', 'C', 'Am', 'Cm', 'C', 'Em', 'Em', 'G', 'Dm', 'A#', 'Em', 'Am', 'G', 'G', 'G', 'Am', 'C', 'C', 'Em', 'Em', 'G', 'Bm', 'C', 'C', 'Am', 'G', 'C', 'Dm', 'G', 'Dm', 'Gm', 'F', 'F', 'C', 'F', 'G', 'F', 'A#', 'Am', 'C', 'G', 'G', 'D#', 'Gm', 'C', 'C', 'Am', 'D#', 'Gm', 'A', 'G', 'Gm', 'C', 'Gm', 'G', 'G', 'A', 'Gm', 'E', 'C', 'C', 'Am', 'Em', 'G', 'C', 'Am', 'Am', 'C', 'G', 'Bm', 'Am', 'A', 'C', 'Em', 'C', 'G', 'Bm', 'Dm', 'F', 'C', 'G', 'Em', 'G', 'D', 'E', 'D', 'D', 'Em', 'C', 'Gm', 'Bm', '

In [13]:
#Question 2
'''
Write a function to load and preprocess a reference annotation (or ground truth) file, saved in CSV
format. The function should take as input the path to a CSV file and produce as output a list of ground
truth chord labels, after suitable pre processing. The output must be a list

λgt = [λgt0, λgt1, ..., λgtN−1] (2)

where each element λgtn is the ground truth chord label for the time window n. 
The length of the list must be adapted to match the the feature rate.

The reference annotations stored in the CSV file are given in the form of labelled segments, each
specified as a triplet (start, end, λ) where start and end are expressed in seconds. To load the CSV file
check the csv library (distributed with Python) or Pandas library (needs to be installed).

In the preprocessing step you should
• convert the segment-based annotation into a frame-based label sequence adapted to the feature rate
used for the chroma sequence;
• convert the labels used in the annotation file to match the chord labels used for the chord recognition
algorithm in terms of enharmonic equivalence (i.e., Db = C# );

• reduce the chord label set used in the annotation to match the chord labels set used for the chord
recognition algorithm (i.e., DMaj6 → D); for this step you can choose any reduction strategy.

Once the function is defined in the notebook, test the function on the CSV file Beatles LetItBe.csv,
available in /data/csv/ folder, and print or plot the output.
Explain in the report each step of the preprocessing phase, focusing in particular on the reduction
strategy of the chord label set.
'''

#Read the CSV file from a path and return the list of chords of the csv adapted to length of the same list of the audio.
def chordsFromCsv(path):
    import csv
    with open(path) as csv_file:
        
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        chords = [] #chords of csv
        i = 0;
        note = ['C','D','E','F','G','A','B']
        
        delta_t_csv = [] #time interval of each chords in seconds of the csv
        for row in csv_reader:
            
            if line_count == 0:
                line_count += 1
            else:
                delta_t_csv.append(row[1]) #extract the time interval of the chord
                if "min" in row[2].split(':'):
                    temp = row[2].split(':')[0]+'m' #change min in m
                else:
                    temp = row[2].split(':')[0] #extract the chord
                chords.append(temp.split('/')[0]) #do the proper cut and add to the list

        for e in chords:
            if 'b' in e:
                chords[i]=note[note.index(chords[i][0])-1]+'#' #change 'b' into '#'
            i += 1   
        
       #For each time interval extract the number of repetitions of each chord in csv list.
        delta_t_wav =  H / Fs #time interval of each frame of the audio    
        chordsCSV_adapted = [] #list of the chords of csv adapted to the length of the list of the chords of the audio       
        nRep_chord = [] #number of repetitions of each chord of the csv
        frame = 0 #counter of the frames
        
        for i in range(len(delta_t_csv)):
            nRep_float = (float(delta_t_csv[i]) / delta_t_wav) - frame #extract number of repetitions in a frame of the audio
            nRep_chord.append(int(math.ceil(nRep_float))) #always round for excess
            frame = (frame + nRep_chord[i]) #increment the frame
            
        for i in range(len(chords)):
            for j in range(nRep_chord[i]):    
                chordsCSV_adapted.append(chords[i]) #insert in the list the chord for its number of repetitions
        
        #Adapt the length of two lists by removing the last element from the longest one        
        if (len(chordsCSV_adapted) != len(chords_WAV)): 
            diff = np.abs(len(chordsCSV_adapted) - len(chords_WAV))
            for i in range(diff):
                if(len(chordsCSV_adapted) > len(chords_WAV)):
                    chordsCSV_adapted.pop() 
                elif (len(chords_WAV) > len(chordsCSV_adapted)):
                    chords_WAV.pop()
            
    return chordsCSV_adapted

audioCSV = os.path.join('data','csv', 'Beatles_LetItBe.csv') 
chords_CSV = chordsFromCsv(audioCSV);
print('CSV: ' '\033[1m' + 'Beatles_LetItBe.csv' + '\033[0m')
print('List of chords of the CSV file: ' + str(len(chords_CSV))+ ' elements.\n')
print(chords_CSV) 

CSV: Beatles_LetItBe.csv
List of chords of the CSV file: 243 elements.

['C', 'C', 'G', 'G', 'Am', 'A', 'F', 'C', 'C', 'G', 'G', 'F', 'C', 'C', 'C', 'C', 'G', 'Am', 'A', 'F', 'C', 'C', 'G', 'F', 'F', 'C', 'C', 'C', 'G', 'G', 'Am', 'F', 'F', 'C', 'G', 'G', 'F', 'F', 'C', 'Am', 'Am', 'E', 'E', 'F', 'C', 'C', 'C', 'C', 'G', 'F', 'F', 'C', 'C', 'C', 'G', 'G', 'Am', 'F', 'F', 'C', 'C', 'G', 'F', 'F', 'C', 'C', 'C', 'G', 'G', 'Am', 'F', 'F', 'C', 'C', 'G', 'G', 'F', 'C', 'C', 'Am', 'Am', 'E', 'F', 'F', 'C', 'C', 'C', 'G', 'G', 'F', 'F', 'C', 'Am', 'Am', 'E', 'E', 'F', 'C', 'C', 'C', 'C', 'G', 'F', 'F', 'C', 'C', 'F', 'C', 'C', 'G', 'F', 'C', 'C', 'F', 'C', 'C', 'G', 'F', 'C', 'C', 'C', 'G', 'G', 'Am', 'A', 'F', 'F', 'C', 'G', 'G', 'F', 'F', 'C', 'C', 'C', 'G', 'G', 'Am', 'Am', 'F', 'C', 'C', 'G', 'G', 'F', 'C', 'C', 'Am', 'Am', 'E', 'E', 'F', 'C', 'C', 'C', 'C', 'G', 'G', 'F', 'F', 'C', 'C', 'C', 'G', 'G', 'Am', 'A', 'F', 'C', 'C', 'G', 'G', 'F', 'F', 'C', 'C', 'C', 'G', 'G', 'Am', 'A', 'F',

In [16]:
#Question 3
'''
Propose a metric for evaluating the template based chord recognition algorithm. 
A metric

m = f(λ_pred,λ_gt) (3)

is a scalar number that expresses how good is the algorithm in performing the task of chord recognition.
The proposed metric should have higher values when the chord recognition algorithm is able to predict
correctly the ground truth chords most of the times, lower values if the chord recognition algorithm often
fails at recognising the chords.

Write a function that takes as input the list of predicted chord labels, the list of ground truth chord
labels and computes the proposed metric value. The two input lists must have same length and the
output must be a scalar value.

Once the function is defined in the notebook, test the function on the two lists of predicted and ground
truth chord labels computed in Question 1 and Question 2.

In the report write a formal definition of the proposed metric and thoroughly explain the idea behind
the proposal. Can you imagine a musically informed strategy that weights differently mismatch errors of
the chord recognition algorithm?
'''
notes_maj = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'];
notes_min = ['Am', 'A#m', 'Bm', 'Cm', 'C#m', 'Dm', 'D#m','Em', 'Fm', 'F#m', 'Gm', 'G#m'] 
trueChords=chordsFromCsv(path_song);

#recognizedChords=chordList(templates, template_sequence);
#da ultimare

recognizedChords=chords_WAV
#recognizedChords=['F', 'A', 'A', 'D', 'A#', 'A#', 'F', 'A#', 'D#', 'A#', 'F', 'A#', 'A#', 'F', 'Gm', 'A#', 'F', 'A#', 'A#', 'F', 'Gm', 'A#', 'F', 'A#', 'A#', 'F', 'A#', 'D#', 'A#', 'F', 'A#', 'A#', 'F', 'Gm', 'A#', 'F', 'A#', 'A#', 'F', 'Gm', 'A#', 'F', 'A#', 'D#', 'A#', 'A#', 'A#', 'A#', 'D#', 'A#', 'F', 'A#', 'F', 'A#', 'D#', 'A#', 'F', 'A#', 'A#', 'F', 'Gm', 'A#', 'F', 'A#', 'A#', 'F', 'Gm', 'A#', 'F', 'A#', 'D#', 'A#', 'A#', 'A#', 'A#', 'D#', 'A#', 'F', 'A#', 'F', 'A#', 'D#', 'A#', 'F', 'A#', 'A#', 'F', 'Gm', 'A#', 'F', 'A#', 'A#', 'F', 'Gm', 'A#', 'F', 'Gm', 'F', 'F', 'F', 'F', 'A#']
numChords=len(trueChords);
num_of_error=0;
imp_errors=[];

for chord in range(len(trueChords)):
    #print (chord,trueChords[chord])
    if(trueChords[chord]!=recognizedChords[chord]):
        num_of_error += 1;
        print(chord,trueChords[chord]);
        imp_errors.append(abs(notes.index(trueChords[chord])- notes.index(recognizedChords[chord])));
        #calcolare la quantita dell'errore

if (num_of_error!=0):
    perc_success1=math.trunc((100-((num_of_error/numChords)*100)));
    imp_errors_med=sum(imp_errors)/len(imp_errors);
    perc_success2=math.trunc((100-((imp_errors_med/11)*100)));
    print(perc_success1,perc_success2);
    
else:
    print(100);




5 A
10 G
11 F


ValueError: 'Em' is not in list

In [ ]:
'''
Compute the proposed metric for the remaining 3 songs:
• audio Beatles HereComesTheSun.wav, CSV Beatles HereComesTheSun.csv
• audio Beatles PennyLane.wav, CSV Beatles PennyLane.csv
• audio Beatles ObLaDiObLaDa.wav, CSV Beatles ObLaDiObLaDa.csv
all contained in folders /data/wav/ and /data/csv/. Print or plot the metric values.
'''

In [ ]:
'''
Analyse how algorithm parameters affect the performance of the templated based chord recognition
algorithm.
Given one algorithm parameter (i.e., smoothing filter length L), choose a range of 3 possible values
for it (i.e.,L = [0, 10, 20] ). 
For each value of the parameter, compute the predicted labels and the correspondent metric value for each song.
This produces a list of 3 metric values for each song.

Plot the results for all songs in a figure where the x-axis corresponds to the parameter values, the
y-axis corresponds to metric values. An example of the plot is in Figure 1.

Repeat this for at least 3 different algorithm parameters. 
Note that in every experiment you need to change only one parameter value at a time, while the others must be fixed.

Show the 3 plots both in the Jupyter Notebook and in the report. 
What considerations can you do from the 3 plots that you have? 
Are some algorithm parameters affecting the results more than others?
Comment the results in the report.
'''